In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import os, sys
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))), "lib/python"))

import numpy as np

from picopic.plot_builder import PlotBuilder
from picopic.h5_reader import H5Reader
from picopic.plain_reader import PlainReader

In [ ]:
##  configuration options
data_path = '../simulation_result'

timestamp=1e-11
show_grid=False
use_cache=False
cmap='terrain'
image_interpolation = 'nearest'
autoselect = True

cmp = 'E_r' # component
specie = 'None'

if os.path.isfile(os.path.join(data_path, "metadata.json")):
    reader = PlainReader(path = data_path, use_cache=use_cache, verbose=True)
elif os.path.isfile(os.path.join(data_path, "data.h5")):
    reader = H5Reader(path = data_path, use_cache=use_cache, verbose=True)
else:
    raise EnvironmentError("There is no corresponding data/metadata files in the path " + data_path + ". Can not continue.")

# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
size=[0, 0, reader.meta.geometry_grid[0], reader.meta.geometry_grid[1]]

r_scale = (size[2] - size[0]) / reader.meta.geometry_grid[0]
z_scale = (size[3] - size[1]) / reader.meta.geometry_grid[1]

# color limits (WARNING: clim_estimation may works incorrectly)
clim_estimation = reader.meta.get_clim_estimation()
clim = [-clim_estimation, clim_estimation]

x_axis_label = r'$\mathit{Z (m)}$'
y_axis_label = r'$\mathit{R (m)}$'
cbar_axis_label = r'$\frac{V}{m}$'
cbar_bunch_density_axis_label = r'$m^{-3}$'

plot_name = r'$\mathbf{Something\enspace to\enspace Debug}$'

In [ ]:
# define plot builder
plot = PlotBuilder(size[3] - size[1], size[2] - size[0],
                   fig_color=reader.meta.figure_color, 
                   fig_width=reader.meta.figure_width,
                   fig_height=reader.meta.figure_height, 
                   fig_dpi=reader.meta.figure_dpi,
                   font_family=reader.meta.figure_font_family,
                   font_name=reader.meta.figure_font_name,
                   font_size=reader.meta.figure_font_size,
                   
                   x_ticklabel_end=reader.meta.geometry_size[1] * z_scale, y_ticklabel_end=reader.meta.geometry_size[0] * r_scale,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect='equal', image_interpolation=image_interpolation, guess_number_ticks=20)

In [ ]:
# get data
data = []

for probe in reader.meta.probes:
    frame = reader.meta.get_frame_number_by_timestamp(timestamp, probe.schedule)
    if (probe.shape == 'rec') and (probe.size[0] == size[0]) and (probe.size[1] == size[1]) and(probe.size[2] == size[2]) and(probe.size[3] == size[3]):
        if probe.component == cmp: data = reader.get_frame("{}".format(cmp), size, frame)

# try bigger frames, if autoselect enabled
if len(data) == 0 and autoselect:
    for probe in reader.meta.probes:
        frame = reader.meta.get_frame_number_by_timestamp(timestamp, probe.schedule)
        if (probe.shape == 'rec') and (probe.size[0] <= size[0]) and (probe.size[1] <= size[1]) and(probe.size[2] >= size[2]) and(probe.size[3] >= size[3]):
            if probe.component == cmp and len(data) == 0: data = reader.get_frame("{}".format(cmp), probe.size, frame)[size[0]:size[2], size[1]:size[3]]

In [ ]:
# add subplots
plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

# add timestamp to plot
plot.get_figure().suptitle("Time: {:.2e} s".format(timestamp), x=.85, y=.95)

# add images
plot.add_image(plot_name, data, cmap=cmap, clim=clim)

# add colorbars
plot.add_colorbar(plot_name, ticks=clim, title=cbar_axis_label)

plot.show()